# Ensemble fork with weights

## I forked this notebook: https://www.kaggle.com/code/apoorvbhardwaj/amex-early-ensemble/notebook?scriptVersionId=97132599
## Please give that notebook an upvote if you give this one an upvote...

I took the above notebook and played around with model weights. Pretty simple idea added to the above simple idea.

Here's a good explanation of the idea of doing a weighted ensemble.
https://machinelearningmastery.com/weighted-average-ensemble-with-python/


Anyway, just a slight improvement over the origial linked notebook above
* Original score = 0.794
* This score = 0.794 ++ (the leaderboard needs to add some decimal places...)

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px

In [ ]:
targetName = 'prediciton'
competitionDir = '../input/amex-default-prediction'
submission = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')

In [ ]:
import re
regex = "([^\\s]+(\\.(?i)(csv))$)"
p = re.compile(regex)

In [ ]:
preds = []

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy import stats

# Weighted average
As per the little function below, I wrote a little formula for weighting the predictions that is based on how well the model it came from scored. The best model is weighted 1, and the worst is weighted somewhere around 0.05.

The article that's linked below has some more elegant ways to do the weighting if you actually buld the models youself and can therefore do some cross validation, but here, since I'm just pulling the output from other people's hard work, I can't and can just do the crude little thing below. 
https://machinelearningmastery.com/weighted-average-ensemble-with-python/

In [ ]:
#crude formula for model weighting.
# Using this formaula the best model's weight = 1, the worst model's weight ~0.05
def model_weight(model_loss, worst_loss, best_loss):
    return 1- ((best_loss - model_loss)/(best_loss-(worst_loss - 0.01)))

In [ ]:
best = 0.794
worst = 0.783
w1 = model_weight(.79, worst, best)
w2 = model_weight(0.794, worst, best) 
w4 = model_weight(0.791, worst, best)
w6 = model_weight(0.794, worst, best)
w7 = model_weight(0.783, worst, best)
#w8 = model_weight(.715, worst, best)


df = pd.read_csv('../input/amex-lightgbm-quickstart/submission.csv')
df['prediction'] = df['prediction']*w1
preds.append((df['prediction']))
print(np.column_stack(preds).shape)

#df2 = pd.read_csv('../input/amex-catboost-0-793/submission_cat_0.7887909351813577.csv')
df2 = pd.read_csv('../input/amex-catboost-rounding-trick/submission_cat_0.7923765971248049.csv')
df2['prediction'] = df2['prediction']*w2
preds.append((df2['prediction']))
print(np.column_stack(preds).shape)

#df3 = pd.read_csv('../input/fork-of-amex-lightgbm-quickstart/submission.csv')
#preds.append((df3['prediction']))
#print(np.column_stack(preds).shape)

df4 = pd.read_csv('../input/lb-split-downsampling-trick-amex/submission.csv')
df4['prediction'] = df4['prediction']*w4
print(np.column_stack(preds).shape)
preds.append(df4['prediction'])

#df5 = pd.read_csv('../input/amex-lgbm-features-eng/submission.csv')
#preds.append((df5['prediction']))
df6 = pd.read_csv('../input/amex-lightautoml-starter/lightautoml_tabularautoml.csv')
df6['prediction'] = df6['prediction']*w6
preds.append(df6['prediction'])
print(np.column_stack(preds).shape)

df7 = pd.read_csv('../input/amex-default-prediction-keras-starter/my_submission.csv')
df7['prediction'] = df7['prediction']*w7
preds.append(df7['prediction'])
print(np.column_stack(preds).shape)


# df8 = pd.read_csv('../input/best-correlated-features-with-low-correlation/submission.csv')
# df8['prediction'] = df8['prediction']*w8
# preds.append(df8['prediction'])
# print(np.column_stack(preds).shape)

In [ ]:
preds

In [ ]:
%%time
blend_ss = submission.copy()

In [ ]:
blend_ss['prediction'] = (np.sum(np.column_stack(preds), axis=1) / (w1+w2+w4+w6+w7))
blend_ss

In [ ]:
blend_ss.to_csv('submission.csv', index=False)